In [40]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [6]:
!pip install xgboost
!pip install optuna


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 9.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.5/599.5 kB 29.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [41]:
%%time
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns, xgboost as xgb, tensorflow as tf, optuna
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error

CPU times: user 56 μs, sys: 0 ns, total: 56 μs
Wall time: 62 μs


In [42]:
data_train = pd.read_csv('/kaggle/input/playground-series-s5e2/training_extra.csv')
data_test = pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv')
data_train_el = pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv')
data_train_el.head(2)

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056


In [43]:
print(f"{data_train.shape}\n{data_train_el.shape}")

(3694318, 11)
(300000, 11)


In [44]:
strange_cols = ['Brand', 'Material', 'Size', 'Style', 'Color']
for col in strange_cols:
    print(f"{data_train_el[col].value_counts()}\n")

Brand
Adidas          60077
Under Armour    59992
Nike            57336
Puma            56814
Jansport        56076
Name: count, dtype: int64

Material
Polyester    79630
Leather      73416
Nylon        70603
Canvas       68004
Name: count, dtype: int64

Size
Medium    101906
Large      98643
Small      92856
Name: count, dtype: int64

Style
Messenger    100031
Tote          97438
Backpack      94561
Name: count, dtype: int64

Color
Pink     51690
Gray     50280
Blue     48097
Red      47219
Green    46386
Black    46378
Name: count, dtype: int64



In [57]:
for col in strange_cols + ['Laptop Compartment', 'Waterproof']:
    data_train[col] = LabelEncoder().fit_transform(data_train[col])
    data_train_el[col] = LabelEncoder().fit_transform(data_train_el[col])
    
    data_test[col] = LabelEncoder().fit_transform(data_test[col])
data_train_el.head(2)

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,1,1,1,7.0,1,0,2,0,11.611723,112.15875
1,1,0,2,10.0,1,1,1,3,27.078537,68.88056


In [46]:
data_train = data_train.drop('id', axis=1)
data_train_el = data_train_el.drop('id', axis=1)
print('done')

done


In [47]:
count = 0
for col in data_train_el.columns.to_list():
    print(f"{col} : {data_train_el[col].isna().sum()}")
    count += data_train_el[col].isna().sum()
print(f"Total : {count}")

Brand : 0
Material : 0
Size : 0
Compartments : 0
Laptop Compartment : 0
Waterproof : 0
Style : 0
Color : 0
Weight Capacity (kg) : 138
Price : 0
Total : 138


In [58]:
data_train['Weight Capacity (kg)'] = data_train['Weight Capacity (kg)'].fillna(data_train['Weight Capacity (kg)'].mean())
data_train_el['Weight Capacity (kg)'] = data_train_el['Weight Capacity (kg)'].fillna(data_train_el['Weight Capacity (kg)'].mean())

data_test['Weight Capacity (kg)'] = data_test['Weight Capacity (kg)'].fillna(data_test['Weight Capacity (kg)'].mean())

data_train_el['Weight Capacity (kg)'].isna().sum()

np.int64(0)

In [49]:
X_train, X_valid, y_train, y_valid = train_test_split(
    data_train_el.iloc[:, :-1],
    data_train_el.iloc[:, -1],
    test_size = 0.2
)
print('done')

done


In [52]:
model_xgb = xgb.XGBRegressor(
    n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42
)
model_rf = RandomForestRegressor(
    n_estimators=100, max_depth=5, random_state=42
)

ens = VotingRegressor(
    estimators=[('xgb', model_xgb), ('rf', model_rf)]
)

ens.fit(X_train, y_train)
print('done')

done


In [53]:
y_pred = ens.predict(X_valid)
print(mean_squared_error(y_pred, y_valid))

1517.709439395029


In [61]:
data_test.head(2)

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
0,300000,3,1,2,2.0,0,0,2,3,20.671147
1,300001,2,0,1,7.0,0,1,0,3,13.564105


In [62]:
X_test = data_test.iloc[:,1:]
preds_test = ens.predict(X_test)

In [63]:
output = pd.DataFrame({'id': data_test['id'],
                       'Price': preds_test})
output.to_csv('submission.csv', index=False)